In [1]:
# Importing wandb and pandas to fetch and read the artifact sample.csv
import wandb  
import pandas as pd

run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: antoniosfl (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


**Pandas profiling**

In [2]:
# Importing pandas profiling to perform EDA
import pandas_profiling

#Creating profile
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

With the help of pandas profiling we notice a few things:
- there are missing values in a few columns
- the column last_review is a date but it is in string format
- the price column has some zeros and some very high prices. These are outliers that need to be taken care of

In [3]:
# We decide to consider from a minimum of $ 10 to a maximum of $ 350 per night.
min_price = 10
max_price = 350

# Dropping the outliers
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

In [4]:
# Converting last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [5]:
# Checking if changes took place
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

**NOTE:** we are not imputing missing values as this needs to happen in the inference pipeline. This way we will be able to handle missing values also in production.

In [6]:
# Terminating the run
run.finish()